## Quantum Double-slit Experiment: A Monte Carlo Model

Let’s consider the double-slit experiment as an example of the "Monte Carlo" simulation technique.  In the lab, we relate the intensity of the observed beam (either photons or electrons) to the sum of the two waves, one from each slit.  Each slit gives us a diffraction pattern,

$$ 
I_{SS_{diffraction}} = \text{sinc}^2(a x / \lambda L)
$$

where $\text{sinc}(x) = \sin(\pi x)/(\pi x)$ is the [normalized sinc function](http://en.wikipedia.org/wiki/Sinc_function).

The double slit, however, is not just the sum of the two single slits, but rather includes an interference term,

$$
I_{DS_{interference}} = \cos^2(2\pi d x/\lambda L)
$$ 

due to the wave-nature of the photons or electrons. 

The observed intensity includes both the probability that the diffraction and interference are appreciable.

$$
I_{DS_{total}} = I_{SS_{diffraction}} * I_{DS_{interference}}
$$



<table>
<tr>
<td>
Here is a diagram to illustrate the concept and define the variables.
<img src = "../img/QuantumDoubleSlit.png" width = 300>
</td>
<td>The intensity on the screen will look something like this:
<img src = "../img/DSIntensity.png" width = 400></td>
</tr>
</table>

Now, let’s do the quantum mechanics problem.  What if we let just one photon or electron pass through the slit at a time?   What would we see on the screen?  

Instead of seeing the addition of waves, we’d see the location of the *individual* photon or electron.   Because $E = h\nu$, the intensity plotted above is also the un-normalized probability distribution of finding a photon or electron at any single location.  

To simulate this experiment, we’ll define the experimental parameters, create the probability distribution,  and then throw random numbers to distribute photons based on their probability.  To make it awesome, we'll set the parameters up as an interactive widget so we can explore the system in detail.

Let the initial distance between the slits $d$ = 15 $\mu$m and the slit width $a$ = 10 $\mu$m.  The screen is 1 m from the plate with the slits.  We will use a He-Neon laser with wavelength $\lambda$ = 632.8 nm.  The screen size is (0.2 $\times$ 0.2) m.

We can set up the probability distribution for this situation and use a Monte-Carlo simulation technique to generate $N$ photons in the range (0,10000).


In [ ]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plot
from scipy.integrate import trapz,cumtrapz
from IPython.html.widgets import interact, interactive

In [ ]:
def distribute1D(x,prob,N):
    """takes any distribution which is directly proportional 
    to the number of particles, and returns data that is 
    statistically the same as the input data."""
    CDF = cumtrapz(prob)/np.sum(prob)
    xsamples = np.zeros(N,float)
    for i in range(0,N):
        r = np.random.ranf()
        xsamples[i] = x[CDF.searchsorted(r)]
    return xsamples

Now define the `double_slit` function and make it interactive:

In [ ]:
#Quantum double-slit
#define the experimental parameters
#d = 15.  # (micron) dist. between slits
#a = 10.  # (micron) slit width. 
#L = 1.   # (m) dist. from slit to screen
#lam = 632.8  # (nm) He-Neon laser

def double_slit(d=15.,a=10.,L=3.,lam=632.8,N=0):

    #convert d and a in microns to meters
    dm = d*1.e-6
    am = a*1.e-6

    #convert wavelength from nm to m
    wave=lam*1.e-9

    # create the probability distribution
    x = np.linspace(-0.2,0.2,10000)    
    #Isingle = np.sin(np.pi*am*x/wave/L)**2./(np.pi*am*x/wave/L)**2
    Isingle = np.sinc(am*x/wave/L)**2.
    Idouble = (np.cos(2*np.pi*dm*x/wave/L)**2)
    Itot = Isingle*Idouble

    #generate the random photon locations on the screen
    #x according to the intensity distribution
    xsamples = distribute1D(x,Itot,N)
    #y randomly over the full screen height
    ysamples = -0.2 + 0.4*np.random.ranf(N)
    
    #Make subplot of the intensity and the screen distribution
    fig = plt.figure(1,(10,6))
    plt.subplot(2,1,1)
    plt.plot(x,Itot)
    plt.xlim(-0.2,0.2)
    plt.ylim(0.,1.2)
    plt.ylabel("Intensity",fontsize=20)

    plt.subplot(2,1,2)
    plt.xlim(-0.2,0.2)
    plt.ylim(-0.2,0.2)
    plt.scatter(xsamples,ysamples)
    plt.xlabel("x (m)",fontsize=20)
    plt.ylabel("y (m)",fontsize=20)

v5 = interact(double_slit,d=(1.,20.,1.), a=(5,50.,1.), L=(1.0,3.0),
                 lam=(435.,700.),N=(0,10000))

Observe how the interference pattern builds up as more photons are added.  Does it evolve like you'd expect?  What effect does varying each of the parameters have on the shape of the distributions?

---